In [5]:
%matplotlib inline

In [9]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

To read the data file we will split the file into lines, and then split
lines into pairs. 




In [19]:
import csv

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    #lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    
    with codecs.open("arabic_words.csv", "r", encoding='utf-8') as fp:
        reader = csv.reader(fp)
        
        # read rest of file
        for row in reader:
            # Print each row
            print(row)

    
#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]

#     # Reverse pairs, make Lang instances
#     if reverse:
#         pairs = [list(reversed(p)) for p in pairs]
#         input_lang = Lang(lang2)
#         output_lang = Lang(lang1)
#     else:
#         input_lang = Lang(lang1)
#         output_lang = Lang(lang2)

#     return input_lang, output_lang, pairs

In [20]:
readLangs("noDiac", "diac")

Reading lines...
['', '']
['ء', 'ء']
['ءئ', 'ءئ']
['ءة', 'ءة']
['ءح', 'ءح']
['ءد', 'ءد']
['ءر', 'ءر']
['ءزإ', 'ءزإ']
['ءس', 'ءس']
['ءط', 'ءط']
['ءؼ', 'ءؼ']
['ءفد', 'ءفد']
['ءك', 'ءك']
['ءن', 'ءن']
['ءه', 'ءه']
['ءو', 'ءو']
['ءى', 'ءى']
['ء', 'ءً']
['ءامنوا', 'ءَامَنُوا']
['ء', 'ءُ']
['ء', 'ءِ']
['ء', 'ءْ']
['ء', 'ءٓ']
['آ', 'آ']
['آآ', 'آآ']
['آآميين', 'آآميين']
['آأنت', 'آأَنْتَ']
['آؤ', 'آؤ']
['آإ', 'آإ']
['آإة', 'آإة']
['آب', 'آب']
['آباء', 'آباء']
['آباءهم', 'آباءهم']
['آباء', 'آباءً']
['آباء', 'آباءٌ']
['آباءهم', 'آباءَهم']
['آباؤكم', 'آباؤكم']
['آباؤنا', 'آباؤنا']
['آباؤهم', 'آباؤهم']
['آبائكم', 'آبائكم']
['آبائنا', 'آبائنا']
['آبائه', 'آبائه']
['آبائهم', 'آبائهم']
['آبائهن', 'آبائهن']
['آبائهنالشيخ', 'آبائهنالشيخ']
['آبائي', 'آبائي']
['آباد', 'آباد']
['آبادي', 'آبادي']
['آبار', 'آبار']
['آبارخاص', 'آبارخاص']
['آبارا', 'آبارًا']
['آباط', 'آباط']
['آبد', 'آبد']
['آبدة', 'آبدة']
['آبق', 'آبق']
['آبي', 'آبي']
['آباءنا', 'آبَاءَنَا']
['آباءهم', 'آبَاءَهُمْ']
['آباؤكم', 'آبَاؤُكُم']
[

['أجرا', 'أجراً']
['أجرب', 'أجرب']
['أجرة', 'أجرة']
['أجرة', 'أجرةٍ']
['أجرت', 'أجرت']
['أجرته', 'أجرته']
['أجرتها', 'أجرتها']
['أجرت', 'أجرتِ']
['أجرت', 'أجرتْ']
['أجرد', 'أجرد']
['أجرك', 'أجرك']
['أجركم', 'أجركم']
['أجرك', 'أجركِ']
['أجرموا', 'أجرموا']
['أجرنا', 'أجرنا']
['أجرني', 'أجرني']
['أجره', 'أجره']
['أجرها', 'أجرها']
['أجرهسعيد', 'أجرهسعيد']
['أجرهم', 'أجرهم']
['أجروا', 'أجروا']
['أجروها', 'أجروها']
['أجرى', 'أجرى']
['أجري', 'أجري']
['أجريت', 'أجريت']
['أجريت', 'أجريتُ']
['أجرين', 'أجرين']
['أجرينا', 'أجرينا']
['أجر', 'أجرً']
['أجرا', 'أجرًا']
['أجر', 'أجرٌ']
['أجر', 'أجرٍ']
['أجر', 'أجرَ']
['أجره', 'أجرَه']
['أجرها', 'أجرَها']
['أجر', 'أجرُ']
['أجرك', 'أجرُك']
['أجرك', 'أجرُكِ']
['أجره', 'أجرُه']
['أجرها', 'أجرُها']
['أجره', 'أجرِه']
['أجره', 'أجرَّهُ']
['أجز', 'أجز']
['أجزأ', 'أجزأ']
['أجزأت', 'أجزأت']
['أجزأه', 'أجزأه']
['أجزاء', 'أجزاء']
['أجزاء', 'أجزاءَ']
['أجزاؤه', 'أجزاؤه']
['أجزاؤهم', 'أجزاؤهم']
['أجزائه', 'أجزائه']
['أجزائها', 'أجزائها']
['أجزت', 'أجزت']
['أجزل', 'أ

['أدقق', 'أدقق']
['أدق', 'أدقُّ']
['أدق', 'أدقِّ']
['أدكسل', 'أدكسل']
['أدل', 'أدل']
['أدلة', 'أدلة']
['أدلةثم', 'أدلةثم']
['أدلة', 'أدلةٌ']
['أدلة', 'أدلةٍ']
['أدلتها', 'أدلتها']
['أدلتهم', 'أدلتهم']
['أدلت', 'أدلتْ']
['أدلج', 'أدلج']
['أدلجته', 'أدلجته']
['أدلجوا', 'أدلجوا']
['أدلك', 'أدلك']
['أدلكم', 'أدلكم']
['أدلوا', 'أدلوا']
['أدلى', 'أدلى']
['أدلىفيها', 'أدلىفيها']
['أدليرا', 'أدليرا']
['أدلع', 'أدلَعَ']
['أدلة', 'أدلّةٍ']
['أدل', 'أدلَّ']
['أدلة', 'أدلَّة']
['أدلة', 'أدلَّةُ']
['أدلته', 'أدلَّته']
['أدلتها', 'أدلَّتها']
['أدل', 'أدلُّ']
['أدلك', 'أدلُّك']
['أدم', 'أدم']
['أدمت', 'أدمت']
['أدمحمد', 'أدمحمد']
['أدمن', 'أدمن']
['أدمنت', 'أدمنت']
['أدمنت', 'أدمنتُ']
['أدمنها', 'أدمنها']
['أدمنوا', 'أدمنوا']
['أدمنوها', 'أدمنوها']
['أدموه', 'أدموه']
['أدمن', 'أدمَن']
['أدمعه', 'أدمُعه']
['أدناسها', 'أدناسها']
['أدناك', 'أدناك']
['أدناكم', 'أدناكم']
['أدناني', 'أدناني']
['أدناه', 'أدناه']
['أدناها', 'أدناها']
['أدناهم', 'أدناهم']
['أدنبره', 'أدنبره']
['أدندا', 'أدندا']
['أدنو', 'أدنو

['أشرحها', 'أشرحها']
['أشرح', 'أشرحُ']
['أشرس', 'أشرس']
['أشرطة', 'أشرطة']
['أشرطة', 'أشرطةِ']
['أشرع', 'أشرع']
['أشرف', 'أشرف']
['أشرفت', 'أشرفت']
['أشرفت', 'أشرفتُ']
['أشرفه', 'أشرفه']
['أشرفها', 'أشرفها']
['أشرفهم', 'أشرفهم']
['أشرف', 'أشرفُ']
['أشرف', 'أشرفِ']
['أشرق', 'أشرق']
['أشرقت', 'أشرقت']
['أشرقت', 'أشرقتْ']
['أشرق', 'أشرقَ']
['أشرك', 'أشرك']
['أشركاني', 'أشركاني']
['أشركتماني', 'أشركتماني']
['أشركوا', 'أشركوا']
['أشرك', 'أشركَ']
['أشرم', 'أشرم']
['أشرنا', 'أشرنا']
['أشروسنة', 'أشروسنة']
['أشرس', 'أشرَسُ']
['أشرقت', 'أشرَقت']
['أشرك', 'أشرَك']
['أشر', 'أشرُّ']
['أشر', 'أشرِّ']
['أشعار', 'أشعار']
['أشعاره', 'أشعاره']
['أشعارها', 'أشعارها']
['أشعارهم', 'أشعارهم']
['أشعاري', 'أشعاري']
['أشعب', 'أشعب']
['أشعباضحك', 'أشعباضحك']
['أشعباضحكمعأشعبتراجموسير', 'أشعباضحكمعأشعبتراجموسير']
['أشعبمصطفى', 'أشعبمصطفى']
['أشعة', 'أشعة']
['أشعت', 'أشعت']
['أشعث', 'أشعث']
['أشعث', 'أشعثَ']
['أشعر', 'أشعر']
['أشعرت', 'أشعرت']
['أشعره', 'أشعره']
['أشعر', 'أشعرَ']
['أشعر', 'أشعرُ']
['أشعل', 'أشعل

['أغر', 'أغرَّ']
['أغرتهم', 'أغرَّتهم']
['أغزر', 'أغزر']
['أغزوه', 'أغزوه']
['أغزر', 'أغزَر']
['أغسطس', 'أغسطس']
['أغصان', 'أغصان']
['أغصانها', 'أغصانها']
['أغصان', 'أغصانِ']
['أغض', 'أغض']
['أغضب', 'أغضب']
['أغضبت', 'أغضبت']
['أغضبتكم', 'أغضبتكم']
['أغضبتهم', 'أغضبتهم']
['أغضبت', 'أغضبتَ']
['أغضبتني', 'أغضبتَني']
['أغضبته', 'أغضبتُه']
['أغضبته', 'أغضبتِه']
['أغضبه', 'أغضبه']
['أغضبونا', 'أغضبونا']
['أغض', 'أغضُّ']
['أغط', 'أغطُّ']
['أغـ', 'أغـ']
['أغفر', 'أغفر']
['أغفل', 'أغفل']
['أغفلت', 'أغفلت']
['أغفلها', 'أغفلها']
['أغلاط', 'أغلاط']
['أغلاق', 'أغلاق']
['أغلال', 'أغلال']
['أغلاله', 'أغلاله']
['أغلب', 'أغلب']
['أغلبنا', 'أغلبنا']
['أغلبه', 'أغلبه']
['أغلبها', 'أغلبها']
['أغلبهم', 'أغلبهم']
['أغلبهن', 'أغلبهن']
['أغلبية', 'أغلبية']
['أغلبية', 'أغلبيَّة']
['أغلب', 'أغلبُ']
['أغلبهم', 'أغلبُهم']
['أغلفة', 'أغلفة']
['أغلق', 'أغلق']
['أغلقا', 'أغلقا']
['أغلقت', 'أغلقت']
['أغلقتها', 'أغلقتها']
['أغلقته', 'أغلقتُه']
['أغلقه', 'أغلقه']
['أغلقوا', 'أغلقوا']
['أغلقي', 'أغلقي']
['أغلى', 'أغلى'

...

['النهر', 'النهر']
['النهر', 'النهرُ']
['النهضة', 'النهضة']
['النهضةالشيخ', 'النهضةالشيخ']
['النهضةحسن', 'النهضةحسن']
['النهضةخاص', 'النهضةخاص']
['النهضةد', 'النهضةد']
['النهضةمن', 'النهضةمن']
['النهضة', 'النهضةَ']
['النهم', 'النهم']
['النهوض', 'النهوض']
['النهوض', 'النهوضَ']
['النهى', 'النهى']
['النهي', 'النهي']
['النهي', 'النهيَ']
['النو', 'النو']
['النوء', 'النوء']
['النوائب', 'النوائب']
['النواب', 'النواب']
['النوابغ', 'النوابغ']
['النواة', 'النواة']
['النواة', 'النواةَ']
['النواتج', 'النواتج']
['النواحى', 'النواحى']
['النواحي', 'النواحي']
['النوادر', 'النوادر']
['النوادرمحمد', 'النوادرمحمد']
['النوادل', 'النوادل']
['النوادي', 'النوادي']
['النوار', 'النوار']
['النواري', 'النواري']
['النوازل', 'النوازل']
['النوازلإيمان', 'النوازلإيمان']
['النوازلالشيخ', 'النوازلالشيخ']
['النوازلبحوث', 'النوازلبحوث']
['النوازلمحمود', 'النوازلمحمود']
['النواس', 'النواس']
['النواس', 'النواسُ']
['النواصر', 'النواصر']
['النواضح', 'النواضح']
['النواظر', 'النواظر']
['النوافح', 'النوافح']
['النوافذ', 'النوا

['الوسطىاليوم', 'الوسطىاليوم']
['الوسطىجمهورية', 'الوسطىجمهورية']
['الوسطىد', 'الوسطىد']
['الوسطىصحف', 'الوسطىصحف']
['الوسطىمأساة', 'الوسطىمأساة']
['الوسطىمجلة', 'الوسطىمجلة']
['الوسطىموقع', 'الوسطىموقع']
['الوسطىوكالات', 'الوسطىوكالات']
['الوسطي', 'الوسطي']
['الوسطيإسماعيل', 'الوسطيإسماعيل']
['الوسطية', 'الوسطية']
['الوسطيةالشيخ', 'الوسطيةالشيخ']
['الوسطيةد', 'الوسطيةد']
['الوسطيةمفهوم', 'الوسطيةمفهوم']
['الوسطي', 'الوسطيّ']
['الوسطية', 'الوسطيَّة']
['الوسع', 'الوسع']
['الوسنان', 'الوسنان']
['الوسواس', 'الوسواس']
['الوسوسة', 'الوسوسة']
['الوسوسةالشيخ', 'الوسوسةالشيخ']
['الوسيط', 'الوسيط']
['الوسيطة', 'الوسيطة']
['الوسيطد', 'الوسيطد']
['الوسيلة', 'الوسيلة']
['الوسيلة', 'الوسيلةَ']
['الوسيلة', 'الوسيلةُ']
['الوسيلة', 'الوسيلةِ']
['الوسيم', 'الوسيم']
['الوسط', 'الوسَط']
['الوشائج', 'الوشائج']
['الوشاة', 'الوشاة']
['الوشاية', 'الوشاية']
['الوشم', 'الوشم']
['الوشيقري', 'الوشيقري']
['الوصابي', 'الوصابي']
['الوصاة', 'الوصاة']
['الوصال', 'الوصال']
['الوصال', 'الوصالُ']
['الوصال', 'الوصالِ']
[

['الغيوب', 'الْغُيُوبِ']
['الغرارة', 'الْغِرَارَةُ']
['الغنى', 'الْغِنَى']
['الغيبة', 'الْغِيبَةِ']
['الـحمد', 'الْـحَمْدَ']
['الفجر', 'الْفجْر']
['الفراء', 'الْفراء']
['الفضل', 'الْفضل']
['الفائدة', 'الْفَائِدَةَ']
['الفائزون', 'الْفَائِزُونَ']
['الفاتحة', 'الْفَاتِحَةَ']
['الفاتحة', 'الْفَاتِحَةِ']
['الفاتحين', 'الْفَاتِحِينَ']
['الفاجر', 'الْفَاجِرُ']
['الفاجر', 'الْفَاجِرِ']
['الفاجعة', 'الْفَاجِعَةُ']
['الفاجعة', 'الْفَاجِعَةِ']
['الفاحش', 'الْفَاحِشَ']
['الفاحشة', 'الْفَاحِشَةَ']
['الفاحشة', 'الْفَاحِشَةُ']
['الفاحشة', 'الْفَاحِشَةِ']
['الفاحش', 'الْفَاحِشِ']
['الفارسي', 'الْفَارِسِيُّ']
['الفارسي', 'الْفَارِسِيِّ']
['الفاسدة', 'الْفَاسِدَةِ']
['الفاسد', 'الْفَاسِدُ']
['الفاسقون', 'الْفَاسِقُون']
['الفاسقون', 'الْفَاسِقُونَ']
['الفاسقين', 'الْفَاسِقِينَ']
['الفاصلين', 'الْفَاصِلِينَ']
['الفاضل', 'الْفَاضِل']
['الفتى', 'الْفَتَى']
['الفتح', 'الْفَتْح']
['الفتح', 'الْفَتْحُ']
['الفتح', 'الْفَتْحِ']
['الفترة', 'الْفَتْرَة']
['الفترة', 'الْفَتْرَةِ']
['الفجر', 'الْفَجْرَ']
['الفجر', 

['انتظرتك', 'انتظرتُك']
['انتظرني', 'انتظرني']
['انتظم', 'انتظم']
['انتظمت', 'انتظمت']
['انتعاش', 'انتعاش']
['انتعاشها', 'انتعاشها']
['انتعش', 'انتعش']
['انتعشت', 'انتعشت']
['انتعشت', 'انتعشَت']
['انتفاء', 'انتفاء']
['انتفاءه', 'انتفاءَه']
['انتفاؤه', 'انتفاؤه']
['انتفاؤها', 'انتفاؤها']
['انتفائه', 'انتفائه']
['انتفاخ', 'انتفاخ']
['انتفاضة', 'انتفاضة']
['انتفاض', 'انتفاضَ']
['انتفاع', 'انتفاع']
['انتفاعات', 'انتفاعات']
['انتفاعا', 'انتفاعاً']
['انتفاعا', 'انتفاعًا']
['انتفت', 'انتفت']
['انتفع', 'انتفع']
['انتفعت', 'انتفعت']
['انتفعنا', 'انتفعنا']
['انتفعوا', 'انتفعوا']
['انتفى', 'انتفى']
['انتقاء', 'انتقاء']
['انتقاء', 'انتقاءَ']
['انتقاء', 'انتقاءُ']
['انتقائكم', 'انتقائكم']
['انتقائي', 'انتقائي']
['انتقائية', 'انتقائية']
['انتقاد', 'انتقاد']
['انتقادات', 'انتقادات']
['انتقاداته', 'انتقاداته']
['انتقاداتها', 'انتقاداتها']
['انتقاده', 'انتقاده']
['انتقاد', 'انتقادٍ']
['انتقاص', 'انتقاص']
['انتقاصا', 'انتقاصاً']
['انتقاصه', 'انتقاصه']
['انتقاصي', 'انتقاصي']
['انتقاصا', 'انتقاصًا']
['انت

['بأنباء', 'بأنباء']
['بأنبياء', 'بأنبياء']
['بأنتظار', 'بأنتظار']
['بأنتويرب', 'بأنتويرب']
['بأنثى', 'بأنثى']
['بأنحاء', 'بأنحاء']
['بأنس', 'بأنس']
['بأنسابهم', 'بأنسابهم']
['بأنسنةالوحي', 'بأنسنةالوحي']
['بأنشطة', 'بأنشطة']
['بأنصار', 'بأنصار']
['بأنظمة', 'بأنظمة']
['بأنعم', 'بأنعم']
['بأنفسكم', 'بأنفسكم']
['بأنفسنا', 'بأنفسنا']
['بأنفسناد', 'بأنفسناد']
['بأنفسهم', 'بأنفسهم']
['بأنفسهمالشيخ', 'بأنفسهمالشيخ']
['بأنفسهن', 'بأنفسهن']
['بأنفسنا', 'بأنفسِنا']
['بأنفسهم', 'بأنفسِهم']
['بأنفه', 'بأنفه']
['بأنفها', 'بأنفها']
['بأنقى', 'بأنقى']
['بأنك', 'بأنك']
['بأنك', 'بأنكَ']
['بأنك', 'بأنكِ']
['بأنما', 'بأنما']
['بأننا', 'بأننا']
['بأنني', 'بأنني']
['بأنه', 'بأنه']
['بأنها', 'بأنها']
['بأنهم', 'بأنهم']
['بأنهما', 'بأنهما']
['بأنهن', 'بأنهن']
['بأنهن', 'بأنهنَّ']
['بأنه', 'بأنهُ']
['بأنوار', 'بأنوار']
['بأنواع', 'بأنواع']
['بأنواعه', 'بأنواعه']
['بأنواعها', 'بأنواعها']
['بأنواع', 'بأنواعٍ']
['بأنواع', 'بأنواعِ']
['بأني', 'بأني']
['بأنياب', 'بأنياب']
['بأنياب', 'بأنيابٍ']
['بأنين', 'بأنين']

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [35]:
MAX_LENGTH = 5

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [36]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    """
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    """
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#print(random.choice(pairs))

Reading lines...
<_io.TextIOWrapper name='arabic.txt' mode='r' encoding='utf-8'>


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




In [10]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [19]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        print("Training pair", training_pair)
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [20]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
print("input:", input_lang.n_words)

input: 940


In [22]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

print("Embeddings:", encoder1.embedding)

#trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
trainIters(encoder1, attn_decoder1, 1, print_every=5000)

Embeddings: Embedding(940, 256)
Training pair (tensor([[ 6],
        [11],
        [36],
        [15],
        [ 1]]), tensor([[ 2],
        [ 3],
        [23],
        [ 4],
        [ 1]]))


NameError: name 'train' is not defined

In [39]:
evaluateRandomly(encoder1, attn_decoder1)

> je tombe .
= i m falling .
< i m losing . <EOS>

> je suis desole je ne te reconnais pas .
= i m sorry i don t recognize you .
< i m sorry i don t recognize you . <EOS>

> je suis en train de dormir .
= i m sleeping .
< i m talking . <EOS>

> je n ai pas faim du tout .
= i m not at all hungry .
< i m not interested at all . <EOS>

> je ne suis pas encore fatigue .
= i m not tired yet .
< i m not tired yet . <EOS>

> elles sont trop grasses .
= they re too fat .
< they re too fat . <EOS>

> je cherche du travail .
= i m looking for a job .
< i m looking for a job . <EOS>

> il est biologiste .
= he is a biologist .
< he is a journalist . <EOS>

> nous sommes ici .
= we are here .
< we re here . <EOS>

> il est mon parent par alliance .
= he is related to me by marriage .
< he is related to him by marriage . <EOS>



Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [40]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [41]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she s five years younger than than me . <EOS>
input = elle est trop petit .
output = she s too short . <EOS>
input = je ne crains pas de mourir .
output = i m not scared of dying . <EOS>
input = c est un jeune directeur plein de talent .
output = he is a talented young man . <EOS>
